## Sentiment Analysis Amazon Reviews
This dataset contains 3.6 millions reviews in Amazon, this dataset is located in Kaggle a very popular dataset repository. https://www.kaggle.com/bittlingmayer/amazonreviews

The code is pretty self-explanatory but if you have any doubts just send me an email to ig.lloret.l@gmail.com

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, GlobalMaxPool1D, Bidirectional,Input,GlobalAveragePooling1D, BatchNormalization, Conv1D,GRU
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.models import Sequential,Model

import os

import sklearn 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
from xgboost import XGBClassifier

from pandas import read_table

import bz2

import re

import spacy
import textblob
from textblob import TextBlob

from sklearn.feature_extraction.text import TfidfTransformer

import re

import collections
from collections import Counter

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

import wordcloud
from wordcloud import WordCloud

import urllib.request

In [ ]:
train_path= path
test_path= path

In [ ]:
train_file = bz2.BZ2File(train_path)
test_file = bz2.BZ2File(test_path)

In [ ]:
train_file_lines = train_file.readlines()
test_file_lines = test_file.readlines()

In [ ]:
train_file_lines = [x.decode('utf-8') for x in train_file_lines]
test_file_lines = [x.decode('utf-8') for x in test_file_lines]

### Dataset Reduce

In [ ]:
def ReduceDataset(train,reduce=5):
    vector = np.random.choice(len(train), size=len(train)//reduce, replace=False)
    vector=list(vector)
    
    
    sentences=pd.DataFrame(train)
    
    train_sentences = sentences.iloc[vector,:][0]
    
    return train_sentences

train_file_lines = ReduceDataset(train_file_lines, reduce=10)

### Preprocessing

In [ ]:
train_labels=[1 if frase.split(" ")[0] =='__label__1' else 0 for frase in train_file_lines]
test_labels=[0 if frase.split(" ")[0] =='__label__2' else 1 for frase in test_file_lines]


train_sentences=[(frase.split(" ",1)[1].lower()) for frase in train_file_lines]
test_sentences=[frase.split(" ",1)[1].lower() for frase in test_file_lines]

In [ ]:
stop_words= stopwords.words('english')

def remove_stopwords(data):
    sentences=[]
    for text in data:
        text = ' '.join(word for word in text.split(' ') if word not in stop_words)
        sentences.append(text)
    return sentences
            

In [ ]:
test_sentences=[re.sub(r'</?a(?:(?= )[^>]*)?>',' ',frase) for frase in test_sentences]
test_sentences=[re.sub(r'[^\w\s]',' ', frase) for frase in test_sentences] 

train_sentences=[re.sub(r'</?a(?:(?= )[^>]*)?>',' ',frase) for frase in train_sentences]
train_sentences=[re.sub(r'[^\w\s]',' ', frase) for frase in train_sentences] 

In [ ]:
df= pd.DataFrame()
df['data'] = train_sentences
df['labels'] = train_labels

In [ ]:
y_train=to_categorical(train_labels)
y_test=to_categorical(test_labels)

In [ ]:
num_words=10000

tokenize=Tokenizer(num_words=num_words)

tokenize.fit_on_texts(train_sentences)
x_train=tokenize.texts_to_sequences(train_sentences)
x_test= tokenize.texts_to_sequences(test_sentences)

In [ ]:
max_len=115

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

x_train_pad=pad_sequences(x_train, maxlen= max_len,padding= "post")
x_test_pad=pad_sequences(x_test, maxlen= max_len, padding="post")

### EDA

In [ ]:
def NumWords(data):
    Numero_palabras=[]
    for frase in data:
        num_words=len(frase.split(" "))
        Numero_palabras.append(num_words)
    return Numero_palabras

def NumChar(data):
    num_character=[]
    for frase in data:
        num_char=len(frase)
        num_character.append(num_char)
    return num_character



Numero_palabras= NumWords(train_sentences)
Number_character= NumChar(train_sentences)

In [ ]:
gr=sns.boxplot(data=Numero_palabras)
gr.set_title("Number of Words")
plt.show()

In [ ]:
gr=sns.boxplot(data=Number_character)
gr.set_title("Number of Characters")
plt.show()

In [ ]:
vocabulary= Counter()
for frase in train_sentences:
    vocabulary.update(list(frase.split()))

In [ ]:
wc = WordCloud(
    background_color='white', 
    max_words=100)

wc.generate(' '.join(text for text in df.loc[df['labels'] == 1, 'data']))


plt.figure(figsize=(15,20))
plt.imshow(wc)
plt.axis('off')
plt.title("Negative Review most common words")
plt.show()

In [ ]:
wc = WordCloud(
    background_color='white', 
    max_words=100)


wc.generate(' '.join(text for text in df.loc[df['labels'] == 0, 'data']))


plt.figure(figsize=(15,20))
plt.imshow(wc)
plt.axis('off')
plt.title("Positive Review most common words")
plt.show()

### Model Building
First we will create a LSTM model and then we will create a Bert Model and we will compare each other

For the LSTM model we will use a GloVe embedding

#### Glove

embeddings_dictionary = dict()
embedding_dim = 100

# Load GloVe 100D embeddings
with open("D:/estudios/GloVe/glove.6B.100d", encoding='utf-8') as fp:
    for line in fp.readlines():
        records = line.split()
        word = records[0]
        vector_dimensions = np.asarray(records[1:], dtype='float32')
        embeddings_dictionary [word] = vector_dimensions

In [ ]:
vocab_length = len(tokenize.word_index) + 1
vocab_length

In [ ]:
embedding_matrix = np.zeros((vocab_length, embedding_dim))

for word, index in tokenize.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
        
embedding_matrix

In [ ]:
x_train,x_valid, y_train, y_valid = train_test_split(x_train_pad,y_train)

#### Full Convolutional Network

In [ ]:
def FCNN_NLP():
    input = Input(shape=(max_len,))
    net= (Embedding(input_dim= embedding_matrix.shape[0], output_dim= embedding_matrix.shape[1],weights=[embedding_matrix], input_length=max_len))(input)
    net = Dropout(0.2)(net)
    net = BatchNormalization()(net)
    
    net = Conv1D(32, 7, padding='same', activation='relu')(net)
    net = BatchNormalization()(net)
    net = Conv1D(32, 3, padding='same', activation='relu')(net)
    net = BatchNormalization()(net)
    net = Conv1D(32, 3, padding='same', activation='relu')(net)
    net = BatchNormalization()(net)
    net = Conv1D(32, 3, padding='same', activation='relu')(net)
    net = BatchNormalization()(net)
    
    net = Conv1D(2, 1)(net)
    net = GlobalAveragePooling1D()(net)
    output = tf.keras.activations.sigmoid(net)
    model = tf.keras.models.Model(inputs = input, outputs = output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model
model = FCNN_NLP()
model.summary()
    

In [ ]:
x_train=np.array(x_train)
x_valid=np.array(x_valid)
y_train=np.array(y_train)
y_valid=np.array(y_valid)
x_test_pad= np.array(x_test_pad)
test_labels= np.array(test_labels)

In [ ]:
model_reduce=8
validation_reduce=2

reduce_lr = ReduceLROnPlateau(
    monitor = 'val_loss', 
    factor = 1/3, 
    verbose = 1, 
    patience = 2,
    min_delta=1e-3,
    min_lr=1e-6
)

history = model.fit(x_train, y_train,steps_per_epoch=len(x_train)//model_reduce//32,validation_data=(x_valid,y_valid), validation_steps=len(x_valid)//validation_reduce//32
,epochs=50, batch_size=32, callbacks=[EarlyStopping(patience=5, restore_best_weights=True), ModelCheckpoint("ModeloLSTM_embedd.h5", save_best_weights=True), reduce_lr])

In [ ]:
tf.keras.models.save_model(model, path)

In [ ]:
model.evaluate(x_test_pad,y_test)

In [ ]:
model.summary()

#### LSTM
We will try a LSTM model with GloVe and without it

In [ ]:
def LSTM_model():
    model= Sequential()
    
    model.add(Embedding(vocab_length, 400))
    
    model.add(Bidirectional(LSTM(max_len, return_sequences=True))) ## if we were using a GPU backend we should remove recurrent dropout
    
    model.add(GlobalMaxPool1D())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(max_len, activation="relu"))
    model.add(Dense(max_len, activation="relu"))

    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(max_len, activation="relu"))

    model.add(Dense(2, activation="softmax"))

    return model

model= LSTM_model()

model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

In [ ]:
model_reduce=8
validation_reduce=2

reduce_lr = ReduceLROnPlateau(
    monitor = 'val_loss', 
    factor = 0.1, 
    verbose = 1, 
    patience = 2,
    min_delta=1e-3,
    min_lr=1e-6
)


history = model.fit(x_train, y_train,steps_per_epoch=len(x_train)//model_reduce//32,validation_data=(x_valid,y_valid), validation_steps=len(x_valid)//validation_reduce//32
,epochs=50, batch_size=32, callbacks=[EarlyStopping(patience=5, restore_best_weights=True), ModelCheckpoint("ModeloLSTM_embedd.h5", save_best_weights=True), reduce_lr])

In [ ]:
tf.keras.models.save_model(model, path)

In [ ]:
model.evaluate(x_test_pad,y_test)

#### Glove LSTM

In [ ]:
def Glove_LSTM():
    model= Sequential()
    
    model.add(Embedding(input_dim= embedding_matrix.shape[0], output_dim= embedding_matrix.shape[1],weights=[embedding_matrix], input_length=max_len))
    
    model.add(Bidirectional(LSTM(max_len, return_sequences=True))) ## if we were using a GPU backend we should remove recurrent dropout
    
    model.add(GlobalMaxPool1D())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(max_len, activation="relu"))
    model.add(Dense(max_len, activation="relu"))

    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(max_len, activation="relu"))

    model.add(Dense(2, activation="softmax"))

    return model

model= Glove_LSTM()

model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
    

In [ ]:
model_reduce=8
validation_reduce=2

history = model.fit(x_train, y_train,steps_per_epoch=len(x_train)//model_reduce//32,validation_data=(x_valid,y_valid), validation_steps=len(x_valid)//validation_reduce//32
,epochs=50, batch_size=32, callbacks=[EarlyStopping(patience=5, restore_best_weights=True), ModelCheckpoint("ModeloLSTM_GloVe.h5", save_best_weights=True), reduce_lr])

In [ ]:
tf.keras.models.save_model(model, path)
model.evaluate(x_test_pad,y_test)

In [ ]:
model.summary()

#### GRU

In [ ]:
def GRU_model():
    model= Sequential()
    
    model.add(Embedding(vocab_length, 400))
    
    model.add(Bidirectional(GRU(max_len, return_sequences=True))) ## if we were using a GPU backend we should remove recurrent dropout
    
    model.add(GlobalMaxPool1D())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(max_len, activation="relu"))
    model.add(Dense(max_len, activation="relu"))

    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(max_len, activation="relu"))

    model.add(Dense(2, activation="softmax"))

    return model

model= GRU_model()

model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

In [ ]:
model_reduce=8
validation_reduce=2

history = model.fit(x_train, y_train,steps_per_epoch=len(x_train)//model_reduce//32,validation_data=(x_valid,y_valid), validation_steps=len(x_valid)//validation_reduce//32
,epochs=50, batch_size=32, callbacks=[EarlyStopping(patience=5, restore_best_weights=True,min_delta= 1e-3), ModelCheckpoint("ModeloGRU_embedd.h5", save_best_weights=True), reduce_lr])

In [ ]:
tf.keras.models.save_model(model, path)

In [ ]:
model.evaluate(x_test_pad,y_test)

#### Glove GRU

In [ ]:
def Glove_GRU():
    model= Sequential()
    
    model.add(Embedding(input_dim= embedding_matrix.shape[0], output_dim= embedding_matrix.shape[1],weights=[embedding_matrix], input_length=max_len))
    
    model.add(Bidirectional(GRU(max_len, return_sequences=True)))
    
    model.add(GlobalMaxPool1D())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(max_len, activation="relu"))
    model.add(Dense(max_len, activation="relu"))

    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(max_len, activation="relu"))

    model.add(Dense(2, activation="softmax"))

    return model

model= Glove_LSTM()

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model_reduce=8
validation_reduce=2

history = model.fit(x_train, y_train,steps_per_epoch=len(x_train)//model_reduce//32,validation_data=(x_valid,y_valid), validation_steps=len(x_valid)//validation_reduce//32
,epochs=50, batch_size=32, callbacks=[EarlyStopping(patience=5, restore_best_weights=True, min_delta= 1e-3), ModelCheckpoint("ModeloLSTM.h5", save_best_weights=True), reduce_lr])

In [ ]:
tf.keras.models.save_model(model, path)

In [ ]:
model.evaluate(x_test_pad,y_test)

#### Model evaluation

test_sentences=[re.sub(r'</?a(?:(?= )[^>]*)?>',' ',frase) for frase in test_sentences]
test_sentences=[re.sub(r'[^\w\s]',' ', frase) for frase in test_sentences] 

y_test=to_categorical(test_labels)

x_test= tokenize.texts_to_sequences(test_sentences)
x_test_pad=pad_sequences(x_test, maxlen= max_len, padding="post")

In [ ]:
modelo_NLP_FCNN_GloVe = tf.keras.models.load_model(path)
modelo_NLP_GRU_embedd= tf.keras.models.load_model(path)
modelo_NLP_LSTM_embedd= tf.keras.models.load_model(path)
modelo_NLP_LSTM_GloVe= tf.keras.models.load_model(path)
modelo_NLP_GRU_GloVe= tf.keras.models.load_model(path)

In [ ]:
modelo_NLP_FCNN_GloVe.evaluate(x_test_pad, y_test)

modelo_NLP_GRU_embedd.evaluate(x_test_pad, y_test)

modelo_NLP_LSTM_embedd.evaluate(x_test_pad, y_test)

modelo_NLP_LSTM_GloVe.evaluate(x_test_pad, y_test)

modelo_NLP_GRU_GloVe.evaluate(x_test_pad, y_test)

In [ ]:
sentence1= "The product was fantastic, very useful"
sentence2= "Just a waste of money I don't recommened anyone to buy this book"

In [ ]:
sentence1 = re.sub(r'</?a(?:(?= )[^>]*)?>',' ',sentence1)
sentence2 = re.sub(r'[^\w\s]',' ', sentence2) 

In [ ]:
sentence1=[(sentence1.split(" ",1)[1].lower())]
sentence2=[sentence2.split(" ",1)[1].lower()]

In [ ]:
sentence1=tokenize.texts_to_sequences(sentence1)
sentence2=tokenize.texts_to_sequences(sentence2)

In [ ]:
sentence1=pad_sequences(sentence1, maxlen= max_len,padding= "post")
sentence2=pad_sequences(sentence2, maxlen= max_len, padding="post")

In [ ]:
print(np.argmax(modelo_NLP_FCNN_GloVe.predict(sentence1)))
print(np.amax(modelo_NLP_FCNN_GloVe.predict(sentence1))) ## with 97 % confidence it's a good review

print(np.argmax(modelo_NLP_FCNN_GloVe.predict(sentence2)))
print(np.amax(modelo_NLP_FCNN_GloVe.predict(sentence2))) ## with 98% confidence it's a bad review


## Looks like it works !!